In [3]:
import requests, json
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'



In [4]:
MARKET_INDEX_SOL = 1
##?????
# userAccount = "ECbtn9Y34m6L89EBDMRYxhpyrFgSHEcbELgozpqAvcDY"


PKEY = "7f6d3DWGkNrKTERtW9McbBQCVbZibuHUf7CsdUjJyK1t"

URL_ORDERS_SELF   = f"https://master.api.drift.trade/orders/userAccounts?userPublicKeys={PKEY}&pageIndex=0&pageSize=100&marketType=perp"

## (below), putting public key only allows me to see
# https://beta.drift.trade/overview/history/tradeHistory?userAccount=7f6d3DWGkNrKTERtW9McbBQCVbZibuHUf7CsdUjJyK1t
URL_HISTORY_SELF  = f"https://beta.drift.trade/overview/history/tradeHistory?userAccount={PKEY}"
URL_DEVNET_OB     = "https://master.dlob.drift.trade/orders/json"
# URL_MAINNET_OB     = "https://dlob.drift.trade/orders/json"



In [31]:
from driftpy.clearing_house_user import ClearingHouseUser
from driftpy.clearing_house import ClearingHouse
from solana.keypair import Keypair
from driftpy.constants.config import configs
import os
from anchorpy import Wallet, Provider
from solana.rpc.async_api import AsyncClient

subaccount_id = 0
env = 'devnet'
url = 'https://api.devnet.solana.com'


with open(os.path.expanduser("/home/sjl/.config/solana/key.json"), 'r') as f: secret = json.load(f)
print(secret)
kp = Keypair.from_secret_key(bytes(secret))
print('using public key:', kp.public_key, 'subaccount=', subaccount_id)
config = configs[env]
wallet = Wallet(kp)
connection = AsyncClient(url)
provider = Provider(connection, wallet)
drift_acct = ClearingHouse.from_config(config, provider)
chu        = ClearingHouseUser(drift_acct)

[148, 209, 110, 105, 122, 200, 248, 79, 216, 183, 169, 188, 42, 128, 250, 0, 100, 25, 52, 129, 48, 52, 24, 8, 54, 157, 192, 237, 218, 137, 127, 247, 196, 32, 197, 180, 22, 36, 110, 80, 129, 84, 220, 246, 238, 97, 79, 94, 211, 160, 65, 158, 188, 226, 73, 81, 104, 220, 215, 171, 59, 61, 176, 219]
using public key: ECbtn9Y34m6L89EBDMRYxhpyrFgSHEcbELgozpqAvcDY subaccount= 0
/home/sjl/Projects/drift/driftpy-examples/venv/lib/python3.10/site-packages/driftpy/idl/drift.json


In [55]:
lev = await chu.get_leverage()
print(lev)

27.648383621405646


In [56]:
gl = await chu.get_total_collateral()
print(gl)

42000911652.11021


In [59]:
s = await chu.get_free_collateral()
print(s)

41989304684.28104


In [73]:
pp = await chu.get_user_position(0)
print(pp.base_asset_amount / 1e9)
print(pp.quote_asset_amount / 1e6)

pp

-4.99
117.287404


PerpPosition(last_cumulative_funding_rate=-3291315921, base_asset_amount=-4990000000, quote_asset_amount=117287404, quote_break_even_amount=117125194, quote_entry_amount=116961035, open_bids=0, open_asks=0, settled_pnl=0, lp_shares=0, last_base_asset_amount_per_lp=0, last_quote_asset_amount_per_lp=0, remainder_base_asset_amount=0, market_index=0, open_orders=0, padding=ListContainer([0]))

In [45]:
r = requests.get(URL_DEVNET_OB).json()
orders_all = list(map(lambda d: {**d, **d.pop('order')}, r['orders']))
df = pd.DataFrame(orders_all)
df[df.user == PKEY]

,user,status,orderType,marketType,slot,orderId,userOrderId,marketIndex,price,baseAssetAmount,...,triggerPrice,triggerCondition,existingPositionDirection,postOnly,immediateOrCancel,oraclePriceOffset,auctionDuration,auctionStartPrice,auctionEndPrice,maxTs
88,7f6d3DWGkNrKTERtW9McbBQCVbZibuHUf7CsdUjJyK1t,open,limit,perp,212233003,73,0,0,0,240000000,...,0,above,short,True,False,-1000000,0,0,0,0


In [46]:
# def get_order_book(market_index=0):

try:
    r = requests.get(URL_DEVNET_OB).json()
except:
    print(r)
    raise Exception("Unable to fetch orderbook")

slot    = r['slot']
oracles = r['oracles']
orders  = r['orders']

    
## market index -> price
market_to_oracle_map = pd.DataFrame(r['oracles']).set_index('marketIndex').to_dict()['price']
market_to_oracle_map
    
    
## Get list of outstanding orders
orders_all = list(map(lambda d: {**d, **d.pop('order')}, orders))
df = pd.DataFrame(orders_all)
df_raw = pd.DataFrame(orders_all)

## Only interested in (a) SOL-PERP (b) limit orders
# SOL is marketINdex=1 as per protocol-v2/sdk/src/constants/spotMarkets.ts
df = df[(df.orderType == 'limit' ) & (df.marketIndex == MARKET_INDEX_SOL) & (df.marketType=='perp')]

df["oraclePrice"] = df["marketIndex"].apply(lambda x: market_to_oracle_map.get(x, None))

## Convert from lamports:
for col in ['price', 'oraclePrice', 'oraclePriceOffset']:
    df[col] = df[col].astype(int)
    df[col] /= 1e6

for col in ['quoteAssetAmountFilled']:
    df[col] = df[col].astype(int)
    df[col] /= 1e6 

for col in ['baseAssetAmount', 'baseAssetAmountFilled']:
    df[col] = df[col].astype(int)
    df[col] /= 1e9

df

,user,status,orderType,marketType,slot,orderId,userOrderId,marketIndex,price,baseAssetAmount,...,triggerCondition,existingPositionDirection,postOnly,immediateOrCancel,oraclePriceOffset,auctionDuration,auctionStartPrice,auctionEndPrice,maxTs,oraclePrice
1,GVZdE1t9jMKZ4rJQZcboy2HxkUjvJAe4abc9wy9fRX5z,open,limit,perp,197947972,115,0,1,19000.0,1.0000,...,above,long,False,False,0.000000,0,0,0,281474976710656,29374.464219
23,CL8wiAZqUoc3Uxo1JaEoRkambR42xwyCvojKtTRigKjx,open,limit,perp,208875895,1538690,0,1,0.0,2.1695,...,above,short,True,False,-26.387281,0,0,0,0,29374.464219
29,CL8wiAZqUoc3Uxo1JaEoRkambR42xwyCvojKtTRigKjx,open,limit,perp,208875895,1538691,0,1,0.0,3.2543,...,above,short,True,False,-41.542083,0,0,0,0,29374.464219
30,CL8wiAZqUoc3Uxo1JaEoRkambR42xwyCvojKtTRigKjx,open,limit,perp,208875895,1538692,0,1,0.0,4.3391,...,above,short,True,False,-56.696886,0,0,0,0,29374.464219


In [ ]:

mdf = df[['price', 'baseAssetAmount', 'direction', 'user', 'status', 'orderType', 'marketType', 'slot', 'orderId', 'userOrderId',
'marketIndex',  'baseAssetAmountFilled',
'quoteAssetAmountFilled',  'reduceOnly', 'triggerPrice',
'triggerCondition', 'existingPositionDirection', 'postOnly',
'immediateOrCancel', 'oraclePriceOffset', 'auctionDuration',
'auctionStartPrice', 'auctionEndPrice', 'maxTs', 'oraclePrice', "user"]]
mdf = mdf.sort_values('price').reset_index(drop=True)


mdf

# Below is load_realtime_book